In [1]:
import os
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output 
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

from models.mlp import MLP  
from models.mlpContrastive import MLPContrastive
from trainer.train import Trainer
from losses.puLoss import PULoss
from dataTools.mnist import MNIST_Chainer, load_dataset, PU_MNIST, PN_MNIST


# 加载数据集

In [2]:
SEED = 0
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
XYtrain, XYtest, prior = load_dataset("cifar10", 10000, 50000)
prior = torch.tensor(prior)
batch_size = 1024

dataset = {'train': MNIST_Chainer(XYtrain),
           'valid': MNIST_Chainer(XYtest)}           
dataloader = {'train': DataLoader(dataset['train'], batch_size= batch_size, shuffle= True, drop_last= True, **kwargs),       # drop_last= True
              'validtrain': DataLoader(dataset['train'], batch_size= batch_size, shuffle= False, **kwargs),
              'valid': DataLoader(dataset['valid'], batch_size= batch_size, shuffle= False, **kwargs)}

# print(prior)
lr = 0.01 #0.0001
n_epochs   = 200
kwargs2 = {
          'train_Dataloader': dataloader['train'],
          'valid_Dataloader': dataloader['valid'],
          'validtrain_Dataloader': dataloader['validtrain'],
          'epochs': n_epochs,   
          'notebook': True,        
          }
# print(kwargs2)


(60000, 1, 28, 28)
training:(60000, 1, 28, 28) consist of 10000 labeled positive samples and 50000 unlabeled samples
test:(10000, 1, 28, 28)


In [2]:
root = '/root/project/biomed-clip-puNCE/Reproduce/mynnPU/data'
SEED = 0
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
batch_size =  30000

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])

pu_mnist = PU_MNIST(root, n_labels=10000, train=True, transform=transform, download=False)
pn_mnist = PN_MNIST(root, train=False, transform=transform, download=False)
dataset = {'train': pu_mnist,
           'valid': pn_mnist}  
prior = pu_mnist.get_prior()

dataloader = {'train': DataLoader(dataset['train'], batch_size= batch_size, shuffle= True, drop_last= True, **kwargs),
              'validtrain': DataLoader(dataset['train'], batch_size= batch_size, shuffle= False, **kwargs),
              'valid': DataLoader(dataset['valid'], batch_size= batch_size, shuffle= False, **kwargs)}

lr = 0.001 #0.0001
n_epochs   = 100
kwargs2 = {
          'train_Dataloader': dataloader['train'],
          'valid_Dataloader': dataloader['valid'],
          'validtrain_Dataloader': dataloader['validtrain'],
          'epochs': n_epochs,   
          'notebook': True,        
          }

## uPU

In [ ]:
model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= lr, weight_decay=0.005)

trainer_uPU  = Trainer('uPU', 
                    model,
                    device, 
                    PULoss(prior= prior, nnPU= False),
                    prior,
                    optimizer,
                    lr_scheduler = None,
                    **kwargs2)

In [ ]:
trainer_uPU.run_trainer()
clear_output()

In [ ]:
trainer_uPU.run_validate('/root/project/biomed-clip-puNCE/Reproduce/mynnPU/checkpoints/checkpoint_20230711213240.pth')

# nnPU

In [3]:
model = MLPContrastive().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= lr, weight_decay=0.005)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=kwargs2["epochs"])

trainer_nnPU  = Trainer('nnPU', 
                    model,
                    device, 
                    PULoss(prior= prior, nnPU= True),
                    prior,
                    optimizer,
                    lr_scheduler = scheduler,
                    **kwargs2)

In [13]:
trainer_nnPU.run_trainer()
print(trainer_nnPU.criterion.number_of_negative_loss)
clear_output()

In [5]:
trainer_nnPU.run_validate('/root/project/biomed-clip-puNCE/Reproduce/mynnPU/checkpoints/checkpoint_20230712114826.pth')


Validation:   0%|          | 0/10 [00:00<?, ?it/s]

The accuracy of this checkpooint is 0.9555


/root/anaconda3/envs/dist-pu/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/root/anaconda3/envs/dist-pu/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
